In [ ]:
include("../Engine/Engine.jl")
using .Engine

using StatsBase, DataFrames
using Graphs, SimpleWeightedGraphs
import PyPlot as plt
import Seaborn as sns
import PlotlyJS

rcParams = plt.PyDict(plt.matplotlib."rcParams")
rcParams["text.usetex"] = false

In [ ]:
cop26 = load_dataset(COP26)
ranks = get_all_ranks(cop26, cop_26_dates)

graphs, cascades, df = load_data("/home/ubuntu/Thesis/Results/JDD_all_users/COP26/data.jld2")
graphs2, cascades2, df2 = load_data("/home/ubuntu/Thesis/Results/JDD_all_users/Random/data.jld2");

In [ ]:
edge_type = "Any Edge"
N_biggest = 20
cuttoff = 0

between, between_actors = betweenness_centralities(graphs, df, cuttoff=cuttoff, edge_type=edge_type)
in_, in_actors = indegree_centralities(graphs, df, cuttoff=cuttoff, edge_type=edge_type)
out, out_actors = outdegree_centralities(graphs, df, cuttoff=cuttoff, edge_type=edge_type)

between_actors[3][1:N_biggest]

In [ ]:
plot_edge_types([graphs, graphs2], [df, df2], [0,0]; y="count_normalized", hue_order=["Before COP26", "During COP26", "After COP26", "Control"], save=true, filename="/home/ubuntu/Thesis/test.png", log=true)

In [ ]:
plot_actors_per_level(cascades, df, control=cascades2)

In [ ]:
cascades[3]
PlotlyJS.plot(plot_cascade_sankey(cascades[3][5], df)...)
# PlotlyJS.savefig(PlotlyJS.plot(plot_cascade_sankey(cascades[3][5], df)...), "/home/ubuntu/Thesis/foo.png")